# Compile Tensorflow Object Detection model for Jetson Nano

## Download Model

In [ ]:
!mkdir -p posenet

In [ ]:
!wget https://storage.googleapis.com/download.tensorflow.org/models/tflite/posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite -O posenet/posenet_mobile.tflite

In [ ]:
model_filename = 'posenet/posenet_mobile.tflite'
model_name = 'posenet-mobile'

# Compress model into .tar.gz so SageMaker Neo can use it
model_tar = model_name + '.tar.gz'
!tar -czf {model_tar} {model_filename}

## Upload Model to S3

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import botocore
import json

In [ ]:
role = get_execution_role()
sess = sagemaker.Session()
region = 'ap-northeast-1'

In [ ]:
# S3 bucket and folders for saving model artifacts.
# Feel free to specify different bucket/folders here if you wish.
bucket = '<your bucket here>'
folder = 'edge-cv-datalake-demo/models/pose-estimator'
training_output_folder = folder + '/training-output'
compilation_output_sub_folder = folder + '/compilation-output'
s3_packaging_output_location = folder + '/packaging-output'

# S3 Location to save the model artifact after compilation
s3_training_output_location = 's3://{}/{}'.format(bucket, training_output_folder)
s3_compilation_output_location = 's3://{}/{}'.format(bucket, compilation_output_sub_folder)
s3_packaging_output_location = 's3://{}/{}'.format(bucket, s3_packaging_output_location)
print(s3_compilation_output_location)
print(s3_packaging_output_location)

In [ ]:
sagemaker_client = boto3.client('sagemaker', region_name=region)

In [ ]:
pose_estimator_s3_path = sess.upload_data(model_tar, bucket, training_output_folder)

In [ ]:
## Update version
pose_estimator_model_version = '0.1.0'

## Start a Neo Compilation Job

In [ ]:
import time

def compile_model(model_type, model_path, model_framework, model_data_shape, target_device, compiler_options):
    compilation_job_name = 'edge-cv-datalake-demo-'+ model_type + '-' + pose_estimator_model_version.replace('.', '-') + '-' + target_device.replace('_', '-')
    print('Compilation job for %s started' % compilation_job_name)

    response = sagemaker_client.create_compilation_job(
            CompilationJobName=compilation_job_name,
            RoleArn=role,
            InputConfig={
                'S3Uri': model_path,
                'DataInputConfig': model_data_shape,
                'Framework': model_framework.upper()
            },
            OutputConfig={
                'S3OutputLocation': s3_compilation_output_location,
                'TargetDevice': target_device,
                'CompilerOptions': compiler_options
            },
            StoppingCondition={
                'MaxRuntimeInSeconds': 1800
            }
        )

    print(response)

    # Poll every 30 sec
    while True:
        response = sagemaker_client.describe_compilation_job(CompilationJobName=compilation_job_name)
        if response['CompilationJobStatus'] == 'COMPLETED':
            break
        elif response['CompilationJobStatus'] == 'FAILED':
            raise RuntimeError('Compilation failed')
        print('Compiling ...')
        time.sleep(30)
    print('Done!')
    return compilation_job_name

In [ ]:
def package_model(compilation_job_name, packaged_model_name, model_version):
    model_packaging_job_name=compilation_job_name
    response = sagemaker_client.create_edge_packaging_job(
        RoleArn=role,
        OutputConfig={
            'S3OutputLocation': s3_packaging_output_location,
        },
        ModelName=packaged_model_name,
        ModelVersion=model_version,
        EdgePackagingJobName=model_packaging_job_name,
        CompilationJobName=compilation_job_name,
    )

    print(response)

    # Poll every 30 sec
    while True:
        job_status = sagemaker_client.describe_edge_packaging_job(EdgePackagingJobName=model_packaging_job_name)
        if job_status['EdgePackagingJobStatus'] == 'COMPLETED':
            break
        elif job_status['EdgePackagingJobStatus'] == 'FAILED':
            raise RuntimeError('Edge Packaging failed')
        print('Packaging ...')
        time.sleep(30)
    print('Done!')
    return model_packaging_job_name

In [ ]:
pose_estimator_model_data_shape = '{"sub_2":[1,257,257,3]}'

model_framework = 'tflite'
target_device = 'jetson_nano'

# Compile for JETPACK 4.5.1 on Jetson Nano 
compiler_options = json.dumps({
    'NVIDIA': {
        'gpu_code': 'sm_53',
        'trt-ver': '7.1.3',
        'cuda-ver': '10.2'
    }
})

In [ ]:
pose_estimator_compilation_job_name = compile_model('pose-estimator', pose_estimator_s3_path, model_framework, pose_estimator_model_data_shape, target_device, compiler_options)

In [ ]:
pose_estimator_packaged_model_name = "pose-estimator-model-packaged-nano"
pose_estimator_model_package = '{}-{}.tar.gz'.format(pose_estimator_packaged_model_name, pose_estimator_model_version.replace('.', '-'))
print(pose_estimator_model_package)

In [ ]:
pose_estimator_packaging_job_name = package_model('edge-cv-datalake-demo-pose-estimator-0-1-0-jetson-nano', pose_estimator_packaged_model_name, pose_estimator_model_version)

In [ ]:
pose_estimator_packaging_job_name = package_model(pose_estimator_compilation_job_name, pose_estimator_packaged_model_name, pose_estimator_model_version)

In [ ]:
target_device = 'rasp3b'
compiler_options = '{"":""}'
pose_estimator_compilation_job_name = compile_model('pose-estimator', pose_estimator_s3_path, model_framework, pose_estimator_model_data_shape, target_device, compiler_options)